In [1]:
from ppsim import Simulation, StatePlotter, HistoryPlotter
from dataclasses import dataclass
import dataclasses
import numpy as np
from matplotlib import pyplot as plt
import ipywidgets as widgets
%matplotlib widget

In [36]:
def make_display(sim, state_map = None):
    sp, hp = StatePlotter(state_map = state_map), HistoryPlotter()
    plt.ioff()
    sim.add_snapshot(sp)
#     sim.add_snapshot(hp)
    

    sim.layout = widgets.AppLayout(
        pane_heights = [1,10,1],
        center=sp.fig.canvas
#         right_sidebar=hp.fig.canvas
    )
    plt.ion()
    display(sim.layout)
    def set_yscale(yscale):
        for snap in sim.snapshots:
            if hasattr(snap, 'ax'):
                snap.ax.set_yscale(yscale)
                snap.update()

    yscale_button = widgets.interactive(set_yscale, yscale=widgets.ToggleButtons(options=['linear', 'symlog'], description = 'y scale:'))
    sim.layout.header=yscale_button
    sim.pane_heights = [1,10]
    
def add_time_bar(sim):
    time_bar = widgets.interactive(sim.set_snapshot_time, time=widgets.FloatSlider(min=sim.times[0], max=sim.times[-1], layout=widgets.Layout(width='100%'), step=0.01))
    sim.layout.footer=time_bar
    
def add_index_bar(sim):
    index_bar = widgets.interactive(sim.set_snapshot_index, index=widgets.IntSlider(min=0, max=len(sim.times)-1, layout=widgets.Layout(width='100%'), step=1))
    sim.layout.footer=index_bar

In [41]:
@dataclass(unsafe_hash = True)
class Agent:
    g: int = 0
    log: int = 0
    counter: int = 0
    def __str__(self):
        return str(self.g) + ',' + str(self.log)
    
def stationary(a, b, bound):
    return min(a + 1, bound), 0

def estimate_size(a: Agent, b: Agent, bound):
    a, b = dataclasses.replace(a), dataclasses.replace(b)
    a.g, b.g = min(a.g + 1, bound), 0
    for i in [a,b]:
        if i.log + 1 in (a.g, b.g, a.log, b.log):
            i.log += 1
    return a, b

In [64]:
n = 10 ** 6
sim = Simulation({(0,0): n}, fast_estimate_size, bound = 30)

In [65]:
n = 10 ** 6
sim = Simulation({Agent(): n}, estimate_size, bound = 30)

In [51]:
sim.config_dict

{Agent(g=0, log=1): 320,
 Agent(g=0, log=2): 6433,
 Agent(g=0, log=3): 23008,
 Agent(g=0, log=4): 28459,
 Agent(g=0, log=5): 30164,
 Agent(g=0, log=6): 183351,
 Agent(g=0, log=7): 205363,
 Agent(g=0, log=8): 16186,
 Agent(g=0, log=9): 3555,
 Agent(g=0, log=10): 1562,
 Agent(g=0, log=11): 672,
 Agent(g=0, log=12): 296,
 Agent(g=0, log=13): 149,
 Agent(g=0, log=14): 50,
 Agent(g=0, log=15): 21,
 Agent(g=0, log=16): 13,
 Agent(g=0, log=17): 3,
 Agent(g=0, log=18): 3,
 Agent(g=0, log=20): 1,
 Agent(g=1, log=1): 236,
 Agent(g=1, log=2): 3783,
 Agent(g=1, log=3): 11937,
 Agent(g=1, log=4): 14130,
 Agent(g=1, log=5): 14516,
 Agent(g=1, log=6): 92233,
 Agent(g=1, log=7): 102385,
 Agent(g=1, log=8): 7737,
 Agent(g=1, log=9): 1638,
 Agent(g=1, log=10): 708,
 Agent(g=1, log=11): 286,
 Agent(g=1, log=12): 132,
 Agent(g=1, log=13): 75,
 Agent(g=1, log=14): 36,
 Agent(g=1, log=15): 8,
 Agent(g=1, log=16): 7,
 Agent(g=1, log=17): 5,
 Agent(g=1, log=19): 2,
 Agent(g=2, log=2): 2336,
 Agent(g=2, log=3)

In [67]:
sim.run(30, 0.1)
add_index_bar(sim)

In [66]:
make_display(sim, lambda x: x.g)
add_index_bar(sim)

AppLayout(children=(Canvas(layout=Layout(grid_area='center'), toolbar=Toolbar(toolitems=[('Home', 'Reset origi…

In [9]:
sim.run(100, 0.1)
sim.set_config({0: 1000})
sim.run(100, 0.1)
sim.set_config(sim.config_array * 1000)
sim.run(100, 0.1)
add_index_bar(sim)

In [6]:
sim.set_config(sim.config_array // 1000)

In [47]:
a = np.ones(10, dtype=np.int64) * 1000
a

array([1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000],
      dtype=int64)

In [49]:
a // 340

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int64)

In [19]:
c = 20
n = 10 ** 10
d = 10 ** -3
init_config = {Agent(level = c, fixed = True): int(d * n) , Agent(level = c, fixed = False): int((1 - d) * n)}
sim = Simulation(init_config, detection_epidemic, c = c)

In [20]:
make_display(sim)

AppLayout(children=(Canvas(layout=Layout(grid_area='center'), toolbar=Toolbar(toolitems=[('Home', 'Reset origi…

In [22]:
sim.run(20, 0.1)
add_index_bar(sim)